In [47]:
# Generated synthetic univariate or multivariate time series data
import pandas as pd
import numpy as np
from einops import rearrange

def generate_time_series_data(start_time: str = "2021-01-01 00:00:00", 
                              freq: str = "H", 
                              periods: int = 40, 
                              n_variables: int = 10, 
                              univariate: bool = False) -> pd.DataFrame:
    """
    Generate a univariate or multivariate time series DataFrame.
    
    Args:
        start_time (str): Start datetime.
        freq (str): Pandas frequency string (e.g., 'H' for hourly).
        periods (int): Number of time steps.
        n_variables (int): Number of variables (ignored if univariate=True).
        univariate (bool): Whether to generate univariate (single-column) data.

    Returns:
        pd.DataFrame: Time series data with datetime index.
    """
    index = pd.date_range(start=start_time, periods=periods, freq=freq)

    if univariate:
        data = np.random.randn(periods)
        df = pd.DataFrame(data, index=index, columns=["A"])
    else:
        columns = list("ABCDEFGHIJ")[:n_variables]
        data = np.random.randn(periods, n_variables)
        df = pd.DataFrame(data, index=index, columns=columns)
    
    return df

df_multivariate = generate_time_series_data()
print(df_multivariate.columns.values)
df_univariate = generate_time_series_data(univariate=True)
print(df_univariate.head())

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J']
                            A
2021-01-01 00:00:00 -0.134010
2021-01-01 01:00:00  0.038150
2021-01-01 02:00:00 -0.660522
2021-01-01 03:00:00 -0.126513
2021-01-01 04:00:00  1.135742


In [62]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule

# Model configuration
MODEL = "moirai"  # model name: choose from {'moirai', 'moirai-moe'}
SIZE = "small"  # model size: choose from {'small', 'base', 'large'}
PDT = 60  # prediction length: any positive integer
CTX = 40  # context length: any positive integer
PSZ = 8  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = 8  # batch size: any positive integer
TEST = 8  # test set length: any positive integer



In [65]:
def moirai_maker(MODEL):
    if MODEL == "moirai":
        model = MoiraiForecast(
            module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-{SIZE}"),
            prediction_length=PDT,
            context_length=CTX,
            patch_size=PSZ,
            num_samples=500,
            target_dim=1,
            feat_dynamic_real_dim=0,
            past_feat_dynamic_real_dim=0,
        )
    elif MODEL == "moirai-moe":
        model = MoiraiMoEForecast(
            module=MoiraiMoEModule.from_pretrained(f"Salesforce/moirai-moe-1.0-R-{SIZE}"),
            prediction_length=PDT,
            context_length=CTX,
            patch_size=PSZ,
            num_samples=500,
            target_dim=1,
            feat_dynamic_real_dim=0,
            past_feat_dynamic_real_dim=0,
        )
    return model

In [66]:
# Moirai univariate
model = moirai_maker("moirai")

# Time series values. Shape: (batch, time, variate)
past_target = rearrange(
    torch.as_tensor(df_univariate["A"].to_numpy(), dtype=torch.float32), "t -> 1 t 1"
)
# 1s if the value is observed, 0s otherwise. Shape: (batch, time, variate)
past_observed_target = torch.ones_like(past_target, dtype=torch.bool)
# 1s if the value is padding, 0s otherwise. Shape: (batch, time)
past_is_pad = torch.zeros_like(past_target, dtype=torch.bool).squeeze(-1)

#print("past_target.shape:", past_target.shape)
#print("past_observed_target.shape:", past_observed_target.shape)
#print("past_is_pad.shape:", past_is_pad.shape)
forecast = model(
    past_target=past_target,
    past_observed_target=past_observed_target,
    past_is_pad=past_is_pad,
)


print(np.round(np.median(forecast[0], axis=0), decimals=4),)




[0.7356 0.7296 0.8398 0.8228 0.8183 0.9249 0.9247 1.0396 0.8851 0.8946
 0.9251 1.0319 0.8427 0.9852 1.0042 1.0281 1.1844 1.1218 1.2159 1.3126
 1.313  1.0993 1.2128 1.4248 1.0125 1.1989 1.2793 1.2406 1.0074 1.2241
 1.1024 1.3489 0.8232 1.0335 0.7033 0.9465 0.9468 1.0141 0.8464 1.1265
 0.9715 1.0845 1.0184 1.1195 1.2554 1.2013 1.2561 1.1922 1.082  1.3733
 1.4954 1.5543 1.4468 1.5289 1.6264 1.6079 1.1143 1.     0.9903 1.1653]


In [10]:
# Moirai-MoE univariate
model = moirai_maker("moirai-moe", gluon_uni)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_uni.input)

input_it = iter(test_data_uni.input)
label_it = iter(test_data_uni.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "A"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [-0.19054992, 0.39191222, -0.19052446, -0.19054992, -0.15109418, -0.24739142, 0.28002316, -0.24767973]}
